In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist
import numpy as np
import tqdm

In [2]:
class Net(nn.Module):
    def __init__(self, input_dim=4, hid_dim=128, nlayers=2, out_dim=2):
        super(Net, self).__init__()
        self.model = []
        self.out_dim = out_dim
        inp = input_dim
        for i in range(nlayers):
            self.model.append(nn.Linear(inp, hid_dim))
            self.model.append(nn.ReLU())
            inp = hid_dim
        self.model.append(nn.Linear(inp, out_dim))
        self.model = nn.Sequential(*self.model)

    def forward(self, x):
        return self.model(x)

In [3]:
class PolicyGradient:
    def __init__(self, model, optim, gamma):
        self.model = model
        self.optim = optim
        self.gamma = gamma
    
    def learn(self, data):
        returns = []
        R = 0
        for r in data['reward'][::-1]:
            R = r + self.gamma * R
            returns.append(R)
        
        returns = torch.Tensor(returns[::-1])
        prob = torch.stack(data['prob'])
        self.optim.zero_grad()
        loss = -(torch.log(prob) * returns).mean()
        loss.backward()
        self.optim.step()
    
    def sample_action(self, state):
        state = torch.from_numpy(state).float()
        prob = torch.softmax(self.model(state), dim=0)
        m = dist.Categorical(prob)
        action = m.sample().item()
        return action, prob[action]

In [4]:
class Args:
    lr = 5e-4
    gamma = 0.98
    max_episodes = 1000

In [5]:
args = Args

In [ ]:
env = gym.make('CartPole-v1')
lr = args.lr
gamma = args.gamma
max_episodes = args.max_episodes

n_obs = env.observation_space.shape[0]
n_act = env.action_space.n
model = Net(n_obs, hid_dim=128, nlayers=2, out_dim=n_act)
optimizer = optim.Adam(model.parameters(), lr=lr)
policy = PolicyGradient(model, optimizer, gamma)

In [ ]:
all_score_list = []
for k in range(10):
    env = gym.make('CartPole-v1')
    lr = args.lr
    gamma = args.gamma
    max_episodes = args.max_episodes

    n_obs = env.observation_space.shape[0]
    n_act = env.action_space.n
    model = Net(n_obs, hid_dim=128, nlayers=2, out_dim=n_act)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    policy = PolicyGradient(model, optimizer, gamma)

    score_list = []
    for i in tqdm.tqdm(range(max_episodes)):
        s = env.reset()
        data_buffer = {'reward': [], 'prob': []}
        done = False
        while not done:
            a, prob = policy.sample_action(s)
            next_s, r, done, info = env.step(a)
            data_buffer['reward'].append(r)
            data_buffer['prob'].append(prob)
            s = next_s
        policy.learn(data_buffer)
        score_list.append(sum(data_buffer['reward']))
    
    all_score_list.append(score_list)